In [1]:
using Random, StatsBase, LightGraphs, GraphPlot, GraphRecipes, Plots, LinearAlgebra, DelimitedFiles

#  Analysis of the linear q-voter model on the ring
N - size of system <br>
dx - percentage of initial positive opinions<br>
L - number of simulations for each dx

In [2]:
function voter(N,dx,L)
    
    all=[]
    
    x = 0:dx:1             #vector of positive opinion percentage

    for xi in x
        #println(xi)
        x_prob_time=[]
        
        positives=0             #all positive outcome
        MonteCarloSteps=0       #time
        
        for simulation in 1:L
            #print(simulation)
            MCS=0               #monte carlo step
            
            net = -ones(N)
            for i in 1:floor(Int,N*xi)
                net[i]=1
            end
            shuffle!(net)
            if xi<=0.1
            end

            net_help=copy(net)
            pushfirst!(net_help,net[end]), push!(net_help,net[2])   #creating net of voters
            
            while net_help!=ones(N+2) && net_help!=-ones(N+2)
                MCS+=1
                for n in N
                    Li = rand(2:N+1)                #listener
                    Sp = rand([Li-1,Li+1])          #speaker

                    net_help[Li]=net_help[Sp]       #change

                    if Li == 2
                        net_help[N+2]=net_help[Sp]
                    elseif Li == length(net_help)-1
                        net_help[1]=net_help[Sp]
                    end
                end
            end
            MonteCarloSteps+=MCS
            
            if net_help==ones(N+2)
                positives+=1
            end
        end
        mean_MCS=MonteCarloSteps/L
        push!(x_prob_time,xi)
        push!(x_prob_time,positives/L)
        push!(x_prob_time,mean_MCS)
        push!(all,x_prob_time)
    end
    return all
end

voter (generic function with 1 method)

In [5]:
N=[100,200,300]
dx=0.02
L=100;

In [4]:
for n in N
    v=voter(n,dx,L)
    writedlm("N"*"$n"*".dx"*"$dx"*".L"*"$L"*".txt", v)
end;

In [12]:
data100 = readdlm("N100.dx0.02.L100.txt")
data200 = readdlm("N200.dx0.02.L100.txt")
data300 = readdlm("N300.dx0.02.L100.txt");

In [14]:
function x_prob_time(data)
    x=zeros(51)
    prob=zeros(51)
    time=zeros(51)
    for i in 1:51
        x[i]=data[i,1]
        prob[i]=data[i,2]
        time[i]=data[i,3]
    end
    return x, prob, time
end

x_prob_time (generic function with 1 method)

In [17]:
x100,prob100,time100=x_prob_time(data100)
x200,prob200,time200=x_prob_time(data200)
x300,prob300,time300=x_prob_time(data300);

In [18]:
probs=[prob100,prob200,prob300]
times=[time100,time200,time300];

In [36]:
Plots.scatter(x100,probs,markershape=[:square :circle :utriangle],
    markerstrokewidth=0.5,labels=["N=100" "N=200" "N=300"],markersize = 6,legend=:bottomright,
    xlabel="Percentage of initial positive opinion",
    ylabel="Probability of all positive outcome",
    title = "100 simulations for each initial percentage",
    legendtitle = "Number of voters")
png("probability")

In [37]:
Plots.scatter(x100,times,markershape=[:square :circle :utriangle],
    markerstrokewidth=0.5,labels=["N=100" "N=200" "N=300"],markersize = 6,
    xlabel="Percentage of initial positive opinion",
    ylabel="Average time to reach consensus [MCS]",
    title = "100 simulations for each initial percentage",
    legendtitle = "Number of voters")
png("time")